In [55]:
import pandas as pd 
import unicodedata
import string
import langid

all_letters = list(string.ascii_letters + " .,;'")
all_letters += ['<SOS>', '<EOS>']

def to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


def is_eng(s):
    if s["origin"] == "English":
        if langid.classify(s["name"])[0] == "en": 
            return s
        else:
            s["name"] = ""
            return s
    else:
        return s


file = open("data/names/names.csv", encoding="utf-8")
df = pd.read_csv(file)
df = df.iloc[:, [0, 1, 2]]
df["name"] = df["name"].transform(lambda x: to_ascii(str(x)))
df["name"] = df["name"].transform(lambda x: x.title())
df = df.dropna()
df = df.drop_duplicates()
df = df[(df.name != "")]
df = df[(df.code != "")]
df = df[(df.gender != "")]
df = df[(df.gender != "?")]

origins = {
    "English": ["GB"], 
    "Greek": ["GR"],
    "Germanic": ["DE", "AT", "SE", "NL", "NO", "BE", "DK"],
    "East-Asian": ["CN", "KP", "VN", "JP"],
    "Turkic": ["TR"],
    "Hispanic": ["ES", "PT", "MX", "BR"],
    "Slavic": ["CZ", "UA", "RU"],
    "French": ["FR"],
    "Italian": ["IT"],
    "Arabic": ["SA", "EG", "MA", "SY", "JO"],
    "Hebrew": ["IL"],
    "Japanese": ["JP"],
    "German": ["DE", "AT"],
    "Scandinavian": ["SE", "NO", "DK"]
}

new_df = pd.DataFrame(columns=["name", "origin", "gender"])

for o in origins:
    selected = df[(df.code.isin(origins[o]))]
    selected["origin"] = o 
    selected = selected.drop("code", 1)
    oversample_factor = int(len(df[(df.code.isin(origins["English"]))]) / len(selected))
    for _ in range(oversample_factor):
        new_df = new_df.append(selected)
    # print("Sample size for " + o + " is: " + str(len(new_df[new_df.origin == o])))


print(len(new_df))
new_df = new_df.apply(is_eng, 1)
new_df = new_df[(new_df.name != "")]
print(len(new_df))

new_df = new_df.sample(n=50000)


/var/folders/r3/jwctyx8d0j901yv_rd4__58r0000gn/T/ipykernel_58439/3056861716.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected["origin"] = o
/var/folders/r3/jwctyx8d0j901yv_rd4__58r0000gn/T/ipykernel_58439/3056861716.py:62: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  selected = selected.drop("code", 1)


445042
431904
